## Import necessary libraries for the script.

In [1]:
import os
import cv2
import numpy as np
from skimage.feature import hog
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

## Image Loader Function

The following function loads images from a given folder, converts them to grayscale, resizes them to **64x64**, and assigns a label.

In [2]:
def loader(folder, label):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        # print(img_path)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Convert to grayscale
        if img is None:
            print(f"Warning: Unable to read {img_path}. Skipping...")
            continue
        img = cv2.resize(img, (64, 64))  # Resize to fixed size
        images.append(img)
        labels.append(label)
        # break
    return images, labels

## Dataset Paths

We define the paths to the dataset folders containing images with and without masks.

In [3]:
mask_folder = "./dataset/with_mask"
no_mask_folder = "./dataset/without_mask"

## Loading the Dataset

We use the `loader` function to load images from the dataset.  
- **`mask_folder`** contains images of people wearing masks (label: `1`).  
- **`no_mask_folder`** contains images of people without masks (label: `0`).  

In [4]:
maskImages, maskLabels = loader(mask_folder, 1)
noMaskImages, noMaskLabels = loader(no_mask_folder, 0)

,Warning: Unable to read ./dataset/with_mask\0_0_≈˙◊¢ 2020-02-23 132115.png. Skipping...
,Warning: Unable to read ./dataset/with_mask\0_0_≈˙◊¢ 2020-02-23 132400.png. Skipping...
,Warning: Unable to read ./dataset/with_mask\0_0_≈˙◊¢ 2020-02-24 171804.png. Skipping...
,Warning: Unable to read ./dataset/with_mask\0_0_≈˙◊¢ 2020-02-24 172039.png. Skipping...
,Warning: Unable to read ./dataset/with_mask\0_0_≈˙◊¢ 2020-02-24 202509.png. Skipping...
,Warning: Unable to read ./dataset/with_mask\0_0_≈˙◊¢ 2020-02-24 205216.png. Skipping...
,Warning: Unable to read ./dataset/with_mask\0_0_≈˙◊¢ 2020-02-24 215234.png. Skipping...
,Warning: Unable to read ./dataset/with_mask\0_0_≈˙◊¢ 2020-02-24 215615.png. Skipping...
,Warning: Unable to read ./dataset/with_mask\0_0_≈˙◊¢ 2020-02-24 220536.png. Skipping...
,Warning: Unable to read ./dataset/with_mask\0_0_≈˙◊¢ 2020-02-24 222124.png. Skipping...
,Warning: Unable to read ./dataset/with_mask\0_0_≈˙◊¢ 2020-02-24 224833.png. Skipping...
,Warning: Unable to r

### Preparing the Dataset

We combine the images and labels from both categories into NumPy arrays:  
- `X` contains all images (masked and unmasked).  
- `y` contains the corresponding labels (`1` for mask, `0` for no mask).  

In [5]:
X = np.array(maskImages + noMaskImages)
y = np.array(maskLabels + noMaskLabels)

### HOG Feature Extraction

The following function `hogExtractor` extracts **Histogram of Oriented Gradients (HOG)** features from a list of images.  

#### Function Breakdown:
- Iterates through each image in the dataset.
- Computes HOG features using:
  - `pixels_per_cell = (8, 8)`: Each cell is 8×8 pixels.
  - `cells_per_block = (2, 2)`: Blocks contain 2×2 cells.
  - `feature_vector = True`: Outputs a flattened feature vector.
- Returns a NumPy array of extracted features.

In [6]:
def hogExtractor(images):
    hog_features = []
    for img in images:
        features = hog(img, pixels_per_cell=(8, 8), cells_per_block=(2, 2), feature_vector=True)
        hog_features.append(features)
    return np.array(hog_features)

### Extracting HOG Features

We apply the `hogExtractor` function to extract **HOG (Histogram of Oriented Gradients)** features from the image dataset.

In [7]:
X_features = hogExtractor(X)

### Splitting the Dataset

We split the dataset into **training** and **testing** sets using an 80-20 ratio.

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X_features, y, test_size=0.2, random_state=42)

### Hyperparameter Tuning and Training SVM Model

We use **GridSearchCV** to find the best hyperparameters for an **SVM classifier**.  

In [14]:
param_grid = {'C': [0.1, 1, 10, 100], 'kernel': ['linear', 'rbf']}
grid = GridSearchCV(SVC(), param_grid, cv=5)
grid.fit(X_train, y_train)

print(f"Best parameters: {grid.best_params_}")
svm_model = SVC(**grid.best_params_)
svm_model.fit(X_train, y_train)

Best parameters: {'C': 10, 'kernel': 'rbf'}


SVC(C=10)

### Training a Neural Network (MLP Classifier)

We train a **Multi-Layer Perceptron (MLP) classifier** for mask detection.

In [10]:
nn_model = MLPClassifier(hidden_layer_sizes=(128, 64), max_iter=500)
nn_model.fit(X_train, y_train)

MLPClassifier(hidden_layer_sizes=(128, 64), max_iter=500)

### Making Predictions

We use the trained models to predict labels for the test set.

In [15]:
svm_pred = svm_model.predict(X_test)
nn_pred = nn_model.predict(X_test)

### Evaluating the SVM Classifier

In [16]:

print("SVM Classifier Report:")
print(classification_report(y_test, svm_pred))

SVM Classifier Report:
,              precision    recall  f1-score   support
,
,           0       0.95      0.94      0.95       362
,           1       0.96      0.96      0.96       453
,
,    accuracy                           0.95       815
,   macro avg       0.95      0.95      0.95       815
,weighted avg       0.95      0.95      0.95       815
,


### Evaluating the Neural Network Classifier Results

In [13]:
print("Neural Network Classifier Report:")
print(classification_report(y_test, nn_pred))

Neural Network Classifier Report:
,              precision    recall  f1-score   support
,
,           0       0.91      0.92      0.91       362
,           1       0.93      0.93      0.93       453
,
,    accuracy                           0.92       815
,   macro avg       0.92      0.92      0.92       815
,weighted avg       0.92      0.92      0.92       815
,
